In [17]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd

import random
import math

In [ ]:
def findGridLocation(loc, size, n_xy):
    gridNum = int(loc[0] // (2*size)) + int(n * (loc[1] // (2 * size)))
    return (gridNum)

In [ ]:
def ifIntersept(loc1, loc2, rot1, rot2, size):
    
    if (loc2 - loc1).length > (size*(2**(1/2))):
        return False
    else:
    
        coord_list_1 = []
        coord_list_2 = []
    
        coord_list_1.append([loc1[0]+size/2, loc1[1]+size/2, loc1[2]+size/2])
        coord_list_1.append([loc1[0]+size/2, loc1[1]-size/2, loc1[2]+size/2])
        coord_list_1.append([loc1[0]-size/2, loc1[1]+size/2, loc1[2]+size/2])
        coord_list_1.append([loc1[0]-size/2, loc1[1]-size/2, loc1[2]+size/2])
    
        coord_list_2.append([loc2[0]+size/2, loc2[1]+size/2, loc2[2]+size/2])
        coord_list_2.append([loc2[0]+size/2, loc2[1]-size/2, loc2[2]+size/2])
        coord_list_2.append([loc2[0]-size/2, loc2[1]+size/2, loc2[2]+size/2])

    for coord in coord_list_2:
                coord[0] = coord[0]*math.cos(rot2[2]) - coord(1)*math.sin(rot2[2])
                coord[1] = coord[0]*math.sin(rot2[2]) + coord(1)*math.cos(rot2[2])

            for coord in coord_list_2:
                coord[0] = coord[0]*math.cos(rot2[1]) + coord(1)*math.sin(rot2[1])
                coord[1] = -coord[0]*math.sin(rot2[1]) + coord(1)*math.cos(rot2[1])

            x1_list = []
            y1_list = []
            z1_list = []

            for c in coord_list_1:
                x1_list.append(c[0])
                y1_list.append(c[1])

            x2_list = []
            y2_list = []
            z2_list = []

            for c in coord_list_2:
                x2_list.append(c[0])
                y2_list.append(c[1])
      
            if ( (max(x1_list) >= min(x2_list)) and (min(x1_list) <= max(x2_list))
                and (max(y1_list) >= min(y2_list)) and (min(y1_list) <= max(y2_list)) ):
                    return False
            else:
                return True

In [ ]:
grain_sizes = [0.05]  # для генерации локально

sample_number = 0

for cubeRadius in grain_sizes:

    sample_number += 1

    # min and max values for each axis for the random numbers
    ranges = {
        'x': {'min': 2 * cubeRadius, 'max': 2 * cubeRadius + 1},
        'y': {'min': 2 * cubeRadius, 'max': 2 * cubeRadius + 1},
        'z': {'min': 2 * cubeRadius, 'max': 2 * cubeRadius + 0.05}
    }

    size = (0.05 / cubeRadius ** 3) * (3/ 7)  # Number of cubes

    # find the number of grid division in every scale (n_xy and n_z)
    if 1 % (2 * cubeRadius) == 0:
        n_xy = int(1 / (2 * cubeRadius)) + 2
        n_z = int(0.05 / (2 * cubeRadius)) + 2
    else:
        n_xy = int(1 // (2 * cubeRadius)) + 3
        n_z = int(0.05 // (2 * cubeRadius)) + 3

    div_number = (n_xy ** 2) * n_z

    cubes = []  # Cube coordinates list
    rotations = []  # cube rotations list

    for i in range(div_number):
        cubes.append([])
        rotations.append([])

    # Generates a random number within the axis minmax range
    randLocInRange = lambda axis: ranges[axis]['min'] + random() * (ranges[axis]['max'] - ranges[axis]['min'])

    # Generate a random 3D coordinate
    loc = Vector([randLocInRange(axis) for axis in ranges.keys()])
    # Add coordinate to cube list
    cubes[findGridLocation(loc, cubeRadius, n_xy)].append(loc)

    # Generates a random rotation
    rot = Euler((random(), random(), random()), 'XYZ')
    # Add rotation to rotations list
    rotations[findGridLocation(loc, cubeRadius, n_xy)].append(rot)

    # Add the first cube (others will be duplicated from it)
    # Note: in blender 2.8 size arg is used instead of radius
    bpy.ops.mesh.primitive_cube_add(size=cubeRadius, location=loc, rotation=rot)

    '''cube = bpy.context.scene.objects['Cube']
    mat = bpy.data.materials.get("Material")
    cube.data.materials.append(mat)
'''

In [ ]:
    # Add all other cubes
    c = 0
    while c <= size:
        while True:
            flag = True
            # Generate a random 3D coordinate
            loc = Vector([randLocInRange(axis) for axis in ranges.keys()])
            # Generate a random rotation    
            rot = Euler((random(), random(), random()), 'XYZ')

            gridNum = findGridLocation(loc, cubeRadius, n_xy)  # find the nunber of grid division for location
            nearGrids = []  # list of grid divisions to check
            nearGrids.append(gridNum + 1)
            nearGrids.append(gridNum - 1)

            nearGrids.append(gridNum + n_xy)
            nearGrids.append(gridNum - n_xy)
            nearGrids.append(gridNum + 1 + n_xy)
            nearGrids.append(gridNum - 1 + n_xy)
            nearGrids.append(gridNum + 1 - n_xy)
            nearGrids.append(gridNum - 1 - n_xy)

            nearGrids.append(gridNum + n_xy + n_xy ** 2)
            nearGrids.append(gridNum - n_xy - n_xy ** 2)

            nearGrids.append(gridNum + n_xy ** 2)
            nearGrids.append(gridNum + 1 + n_xy ** 2)
            nearGrids.append(gridNum - 1 + n_xy ** 2)
            nearGrids.append(gridNum + n_xy + n_xy ** 2)
            nearGrids.append(gridNum - n_xy + n_xy ** 2)
            nearGrids.append(gridNum + 1 + n_xy + n_xy ** 2)
            nearGrids.append(gridNum - 1 + n_xy + n_xy ** 2)
            nearGrids.append(gridNum + 1 - n_xy + n_xy ** 2)
            nearGrids.append(gridNum - 1 - n_xy + n_xy ** 2)

            nearGrids.append(gridNum - n_xy ** 2)
            nearGrids.append(gridNum + 1 - n_xy ** 2)
            nearGrids.append(gridNum - 1 - n_xy ** 2)
            nearGrids.append(gridNum + n_xy - n_xy ** 2)
            nearGrids.append(gridNum - n_xy - n_xy ** 2)
            nearGrids.append(gridNum + 1 + n_xy - n_xy ** 2)
            nearGrids.append(gridNum - 1 + n_xy - n_xy ** 2)
            nearGrids.append(gridNum + 1 - n_xy - n_xy ** 2)
            nearGrids.append(gridNum - 1 - n_xy - n_xy ** 2)

            for i in nearGrids:
                if len(cubes[i]) != 0:
                    for l in range(len(cubes[i])):
                        if ifIntersept(loc, cubes[i][l], rot, rotations[i][l], cubeRadius) is False:
                            flag = False
            if flag is True:
                break

        # Add coordinate to cube list
        cubes[gridNum].append(loc)
        rotations[gridNum].append(rot)
        dupliCube = cube.copy()
        dupliCube.location = loc
        dupliCube.rotation_euler = rot

        # bpy.context.scene.objects.link( dupliCube )
        # in blender 2.8 an api change requires to use the collection instead of the scene
        bpy.context.collection.objects.link(dupliCube)
        c += 1

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.show()